**Final Assignment Week 3**

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

**Get Data From Wikipedia**

In [2]:
# specify which URL/web page 
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# store in a dataframe
list_df = pd.read_html(url, header = 0)
df = list_df[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Eliminate 'Not assigned' Boroughs**

In [3]:
df = df[(df.Borough != 'Not assigned')]

**Group Neighborhoods by Borough and Merge on Neighbourhood**

In [4]:
toronto_grouped = df.groupby(['Postal Code', 'Borough']).agg({'Neighbourhood':lambda x: ', '.join(x)}).reset_index()
toronto_grouped.head()


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Solve Question 1**

In [5]:
toronto_grouped.shape

(103, 3)